In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [12]:
import re
import pytest
import joblib
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.exceptions import OneStepAheadValidationWarning
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import bayesian_search_forecaster_multiseries, grid_search_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.preprocessing import RollingFeatures
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error

In [3]:
series_item_sales = pd.read_parquet(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_multi_series_items_sales.parquet")
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series.joblib")
exog_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [14]:
metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
forecaster = ForecasterRecursiveMultiSeries(
            regressor          = LGBMRegressor(random_state=123, verbose=-1),
            lags               = 3,
            encoding           = 'ordinal',
            transformer_series = StandardScaler(),
            transformer_exog   = StandardScaler(),
            weight_func        = None,
            series_weights     = None,
            differentiation    = None,
            dropna_from_series = False,
        )
cv = OneStepAheadFold(
        initial_train_size = "2016-06-01 00:00:00",
)

def search_space(trial):
    search_space  = {
        'alpha': trial.suggest_float('alpha', 1e-3, 1e3, log=True),
        'lags': trial.suggest_categorical('lags', [3, 5]),
    }

    return search_space

results, _ = bayesian_search_forecaster_multiseries(
    forecaster         = forecaster,
    series             = series_dict,
    exog               = exog_dict,
    cv                 = cv,
    search_space       = search_space,
    n_trials           = 5,
    metric             = metrics,
    aggregate_metric   = ["average", "weighted_average", "pooling"],
    return_best        = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = False,
    suppress_warnings = True,
)

In [13]:
param_grid = {'alpha': [0.01, 0.1, 1]}
lags_grid = [2, 4]

results = grid_search_forecaster_multiseries(
                  forecaster          = forecaster,
                  series              = series_dict,
                  param_grid          = param_grid,
                  cv                  = cv,
                  metric              = 'mean_absolute_error',
                  aggregate_metric    = 'weighted_average',
                  levels              = None,
                  exog                = None,
                  lags_grid           = lags_grid,
                  return_best         = False,
                  verbose             = True
              )

6 models compared for 5 level(s). Number of iterations: 6.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/3 [00:00<?, ?it/s]

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U19'), dtype('int64')) -> None

In [19]:
from skforecast.utils import date_to_index_position

In [28]:
cv = OneStepAheadFold(
        initial_train_size = "2020-03-01 00:00:00",
)

In [29]:
n = 1000000

series = pd.Series(
    np.random.rand(n),
    index=pd.date_range(start='2020-01-01', periods=n, freq='h')
)

In [30]:
%%timeit

cv._extract_index(series)

171 ns ± 2.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [31]:
idx = cv._extract_index(series)

In [32]:
%%timeit

initial_train_size = date_to_index_position(
                                 index        = idx, 
                                 date_input   = cv.initial_train_size, 
                                 method       = 'validation',
                                 date_literal = 'initial_train_size'
                             )

326 μs ± 6.18 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
# Create a dict with 1000 pandas series
series_dict = {
    f'series_{i}': pd.Series(
        np.random.rand(n),
        index=pd.date_range(start='2020-01-01', periods=n, freq='h')
    )
    for i in range(1000)
}

In [34]:
%%timeit

cv._extract_index(series_dict)

10.3 ms ± 108 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
idx = cv._extract_index(series_dict)

In [36]:
%%timeit

initial_train_size = date_to_index_position(
                                 index        = idx, 
                                 date_input   = cv.initial_train_size, 
                                 method       = 'validation',
                                 date_literal = 'initial_train_size'
                             )

312 μs ± 5.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
